In [ ]:
import numpy as np
import pandas as pd
import glob
from newspaper import Article
from time import sleep
from tqdm import tqdm
import time
import signal

In [ ]:
all_files = glob.glob("./all/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    df['umb']=filename.split('/')[2].split('.')[0]
    li.append(df)

dfm = pd.concat(li, axis=0, ignore_index=True)

/Users/michalpalinski/anaconda3/envs/myPython/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (0,1,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/michalpalinski/anaconda3/envs/myPython/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dfm['umb'].unique()

array(['all3', 'all0', 'all1', 'all2'], dtype=object)

In [ ]:
def process(row):
    x=row['entities']
    try:
        y=eval(x)
        return y
    except:
        return 'zero' 

In [ ]:
dfm['entities']=dfm.apply(lambda row: process(row), axis=1)

In [ ]:
dfm=dfm[(dfm['entities'].apply(lambda x: 'urls' in str(x))==True)]

In [ ]:
dfm['url']=dfm['entities'].apply(lambda x: x['urls'])
dfm['url']=dfm['url'].apply(lambda x: x[0]['expanded_url'])

In [ ]:
# df=dfm[dfm['umb']=='all0']
df=dfm

In [ ]:
df.shape

(256681, 19)

In [ ]:
df=df.drop_duplicates(subset='url')

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
len(df)

146553

In [ ]:
df=df[df['umb']!='all0']

In [ ]:
len(df)

56912

In [ ]:
df.to_csv('./all_url/all.csv')

In [ ]:
class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

df_arts=pd.DataFrame()

for no, row in tqdm(df[35300:].iterrows(), total=df[35300:].shape[0]):
    # Start the timer. Once 5 seconds are over, a SIGALRM signal is sent.
    signal.alarm(20)    
    # This try/except loop ensures that 
    #   you'll catch TimeoutException when it's sent.
    try:
        # Whatever your function that might hang
        article = Article(row['url'])
        article.download()
        article.parse()
        title=article.title
        text=article.text
        url=row['url']
        
        df_arts=df_arts.append({
                        'url': url, 
                        'title': title,
                        'text': text,
                       }, 
                       ignore_index=True) 
        

    except:
#         print('exception')
        title=np.nan
        text=np.nan
        url=row['url']
        
        df_arts=df_arts.append({
                        'url': url, 
                        'title': title,
                        'text': text,
                       }, 
                       ignore_index=True) 
        
        continue # continue the for loop if function A takes more than 5 second
        
    else:
        # Reset the alarm
        signal.alarm(0)
    
    sleep(1)
    
    if no%100==0:
        print(no)
        df_arts.to_csv('./newspaper1/'+str(no)+'.csv')
        df_arts=pd.DataFrame()

  0%|          | 1/21612 [00:01<11:55:32,  1.99s/it]

35300


  0%|          | 101/21612 [1:43:51<10:43:16,  1.79s/it]    

35400


  1%|          | 200/21612 [1:48:48<12:44:54,  2.14s/it]

...

 99%|█████████▉| 21360/21612 [19:50:54<11:55,  2.84s/it]

146300


100%|█████████▉| 21560/21612 [20:00:46<02:51,  3.30s/it]

146500


100%|██████████| 21612/21612 [20:03:49<00:00,  3.34s/it]


In [ ]:
all_files = glob.glob("./all_url/*.csv")

li = []

for filename in all_files:
    try:
        df = pd.read_csv(filename, index_col=None, header=0,lineterminator='\n')
        li.append(df)
    except:
        print(filename)
        pass

df = pd.concat(li, axis=0, ignore_index=True)